In [1]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.neighbors import NearestNeighbors
from joblib import dump

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9e5f390cdd7e44658f17b1b3228c02b2",
                                                           client_secret="a2d7e42715d54e12a3a4e86ba5d93dc0"))

In [ ]:
"""
Primary:
- id (Id of track generated by Spotify)
Numerical:
- acousticness (Ranges from 0 to 1)
- danceability (Ranges from 0 to 1)
- energy (Ranges from 0 to 1)
- duration_ms (Integer typically ranging from 200k to 300k)
- instrumentalness (Ranges from 0 to 1)
- valence (Ranges from 0 to 1)
- popularity (Ranges from 0 to 100)
- tempo (Float typically ranging from 50 to 150)
- liveness (Ranges from 0 to 1)
- loudness (Float typically ranging from -60 to 0)
- speechiness (Ranges from 0 to 1)
- year (Ranges from 1921 to 2020)
Dummy:
- mode (0 = Minor, 1 = Major)
- explicit (0 = No explicit content, 1 = Explicit content)
Categorical:
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)
- artists (List of artists mentioned)
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)
- name (Name of the song)"""

In [34]:
# reading csv
df = pd.read_csv('suggestor/edited_data.csv')

In [35]:
df

,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0,0.991000,MAMIE SMITH,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,1,0.643000,SCREAMIN' JAY HAWKINS,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,2,0.993000,MAMIE SMITH,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,3,0.000173,OSCAR VELAZQUEZ,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,4,0.295000,MIXE,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,174384,0.009170,"DJ COMBO', 'SANDER-7', 'TONY T",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,174385,0.795000,ALESSIA CARA,0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,174386,0.806000,ROGER FLY,0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,174387,0.920000,TAYLOR SWIFT,0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [36]:
# adding direct url to data set by adding url prefix and id
url = 'http://open.spotify.com/track/' + df['id']
df['url'] = url

In [37]:
df['combined'] = df['name'] + ' - ' +  df['artists']
df = df.drop_duplicates(subset='combined', keep='first').reset_index()

In [38]:
# reordering columns, leaving out ID and release date
df = df[['combined', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

In [39]:
df

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Keep A Song In Your Soul - MAMIE SMITH,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...,1920,0.991000,0.598,168333,0.2240,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,I Put A Spell On You - SCREAMIN' JAY HAWKINS,http://open.spotify.com/track/0hbkKFIJm7Z05H8Z...,1920,0.643000,0.852,150200,0.5170,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,Golfing Papa - MAMIE SMITH,http://open.spotify.com/track/11m7laMUgmOKqI3o...,1920,0.993000,0.647,163827,0.1860,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,True House Music - Xavier Santos & Carlos Gomi...,http://open.spotify.com/track/19Lc5SfJJ5O1oaxY...,1920,0.000173,0.730,422087,0.7980,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,Xuniverxe - MIXE,http://open.spotify.com/track/2hJjbsLCytGsnAHf...,1920,0.295000,0.704,165224,0.7070,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159433,"I giorni - LUDOVICO EINAUDI', 'JOHANNES BORNLÖF",http://open.spotify.com/track/0sZ6HFSulsrOl6VT...,2021,0.976000,0.358,381150,0.1210,0,0.889000,2,0.1350,-25.111,1,0,0.0532,131.808,0.1060
159434,"Nuvole bianche - LUDOVICO EINAUDI', 'JOHANNES ...",http://open.spotify.com/track/2LeqqwzobL5ktfQh...,2021,0.995000,0.297,349200,0.0287,0,0.908000,8,0.0995,-30.008,1,0,0.0564,141.636,0.0678
159435,"Una Mattina - LUDOVICO EINAUDI', 'JOHANNES BOR...",http://open.spotify.com/track/3Glmyv3hbGGTgeR3...,2021,0.995000,0.343,206700,0.0165,0,0.878000,9,0.0774,-30.915,0,0,0.0455,126.970,0.1510
159436,"Night - LUDOVICO EINAUDI', 'JOHANNES BORNLÖF",http://open.spotify.com/track/6QGVWUbmlePAiY5z...,2021,0.988000,0.316,303333,0.0573,0,0.879000,3,0.1200,-24.121,1,0,0.0515,81.070,0.0373


In [47]:
# finding a song I want to use as an input
tay = df[df['combined'] == 'Heather - CONAN GRAY'].sort_values('popularity', ascending=False)
tay

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
18676,Heather - CONAN GRAY,http://open.spotify.com/track/4xqrdfXkTW4T0Rau...,2020,0.584,0.357,198040,0.425,0,0.0,5,0.322,-7.301,1,91,0.0333,102.078,0.27


In [48]:
# target set will be both artist and name
y_set = ['combined', 'url']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

In [49]:
# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', leaf_size =15, n_neighbors=12, n_jobs=-1)
nn.fit(df_data)

NearestNeighbors(algorithm='brute', leaf_size=15, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
                 radius=1.0)

In [50]:
# sample a song(index) from df_data to use as our query point 
input_index = 18676 # Drake - Tootsie slide

# vectorize 
data_vect = [df_data.iloc[input_index].values]
data_vect

[array([ 2.02000e+03,  5.84000e-01,  3.57000e-01,  1.98040e+05,
         4.25000e-01,  0.00000e+00,  0.00000e+00,  5.00000e+00,
         3.22000e-01, -7.30100e+00,  1.00000e+00,  9.10000e+01,
         3.33000e-02,  1.02078e+02,  2.70000e-01])]

In [51]:
# Query Using kneighbors 
neigh_dist, neigh_indices = nn.kneighbors(data_vect)

In [52]:
# top 12 closest data vectors to our reference vector, data_vect
neigh_dist

array([[ 0.        , 25.89499382, 30.14090711, 35.71330886, 38.85572155,
        40.4806412 , 41.43026923, 41.6082808 , 43.6061903 , 46.59346312,
        47.13994234, 47.18757704]])

In [53]:
# these are the corresponding indicies of the most similar vectors 
neigh_indices

array([[ 18676,  17738,  70665,  85352,  31950,  53992,  15156,  35218,
         16772,  15071, 115265,  17668]], dtype=int64)

In [54]:
# breaking down the array to a list
indexs = neigh_indices.flat[0:12].tolist()
indexs

[18676,
 17738,
 70665,
 85352,
 31950,
 53992,
 15156,
 35218,
 16772,
 15071,
 115265,
 17668]

In [55]:
# result metrics
df_data.iloc[indexs]

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
18676,2020,0.584000,0.357,198040,0.425,0,0.000000,5,0.3220,-7.301,1,91,0.0333,102.078,0.270
17738,2014,0.101000,0.720,198053,0.741,0,0.000014,10,0.2890,-6.529,0,80,0.0287,119.896,0.357
70665,2016,0.000247,0.455,198040,0.910,1,0.000000,2,0.1070,-4.464,1,64,0.1660,114.109,0.447
85352,2010,0.057100,0.869,198040,0.700,0,0.414000,2,0.0691,-4.966,1,57,0.0419,103.983,0.983
31950,1994,0.649000,0.556,198040,0.393,0,0.000000,0,0.2920,-10.427,1,63,0.0319,98.226,0.604
53992,2017,0.114000,0.755,198040,0.536,0,0.000000,8,0.0282,-5.605,1,75,0.1200,138.969,0.882
15156,2000,0.013100,0.777,198066,0.725,0,0.008050,0,0.2460,-8.681,1,71,0.0500,116.669,0.529
35218,2012,0.030100,0.859,198053,0.746,1,0.000877,9,0.1690,-3.014,1,64,0.0663,75.002,0.543
16772,2009,0.220000,0.656,198020,0.817,0,0.000000,11,0.3060,-4.296,0,68,0.0435,130.446,0.862
15071,1999,0.053400,0.748,198067,0.939,0,0.000000,0,0.3200,-4.288,0,60,0.0341,104.001,0.960


In [56]:
# result target
df_target.iloc[indexs]

,combined,url
18676,Heather - CONAN GRAY,http://open.spotify.com/track/4xqrdfXkTW4T0Rau...
17738,Dancin (feat. Luvli) - Krono Remix - AARON SMI...,http://open.spotify.com/track/6WkJ2OK163XXS2oA...
70665,Crazy = Genius - PANIC! AT THE DISCO,http://open.spotify.com/track/6V68ItawQkQlZhYI...
85352,The Ghost Inside - BROKEN BELLS,http://open.spotify.com/track/32fmQsWSAaDas2MJ...
31950,Las piedras rodantes - EL TRI,http://open.spotify.com/track/1gd7Q7mxviLWkG5H...
53992,Personal - HRVY,http://open.spotify.com/track/33KJ2xFYONOlIYDB...
15156,I Will Survive - Single Version - GLORIA GAYNOR,http://open.spotify.com/track/7cv28LXcjAC3GsXb...
35218,"I'm Legit - NICKI MINAJ', 'CIARA",http://open.spotify.com/track/11a6GzsfPoLY1bVw...
16772,Boot Scootin' Boogie - BROOKS & DUNN,http://open.spotify.com/track/7Fq9RwQxSn3kW85P...
15071,(You Drive Me) Crazy - BRITNEY SPEARS,http://open.spotify.com/track/1DSJNBNhGZCigg9l...


In [27]:
# creates the model into a pickle file
dump(nn, 'model.joblib', compress=True)

['model.joblib']